<a href="https://colab.research.google.com/github/Poa463/AntiScrapeArt/blob/main/%E3%83%A6%E3%83%BC%E3%82%B6%E3%83%BC%E7%AE%A1%E7%90%86%E7%94%BB%E9%9D%A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 必要なライブラリのインポート
from google.colab import drive
import firebase_admin
from firebase_admin import credentials, auth

# Google Driveをマウント
drive.mount('/content/drive')

# Google Drive内のサービスアカウントキーのパスを指定
service_account_path = "/content/drive/MyDrive/antiscrapeart-firebase-adminsdk-hmkml-b513c9a5f1.json"  # Google Driveに保存したファイルのパス

# Firebaseの初期化（すでに初期化されている場合は再初期化しない）
if not firebase_admin._apps:
    cred = credentials.Certificate(service_account_path)
    firebase_admin.initialize_app(cred)


In [ ]:
import re  # バリデーションのために正規表現を使用

# ユーザー作成関数
def create_user(email, password, display_name=None, phone_number=None):
    """
    Firebaseに新規ユーザーを作成する関数
    email: 新しいユーザーのメールアドレス
    password: 新しいユーザーのパスワード
    display_name: オプションで設定するユーザーの表示名
    phone_number: オプションで設定するユーザーの電話番号（E.164形式）
    """

    # メールアドレスの形式をチェック
    if not re.match(r"[^@]+@[^@]+\.[^@]+", email):
        print("Error: 無効なメールアドレス形式です。")
        return

    # パスワードのバリデーション（8文字以上、英数字混在）
    if len(password) < 8 or not re.search(r"[A-Za-z]", password) or not re.search(r"\d", password):
        print("Error: パスワードは8文字以上で、英字と数字を含む必要があります。")
        return

    # 電話番号のバリデーション（E.164形式）
    if phone_number and not re.match(r"^\+?\d{10,15}$", phone_number):
        print("Error: 電話番号はE.164形式で入力してください（例: +819012345678）。")
        return

    try:
        # 既にユーザーが存在するか確認
        user = auth.get_user_by_email(email)
        print("Error: このメールアドレスのユーザーは既に存在します:", user.uid)
    except auth.UserNotFoundError:
        # ユーザーが存在しない場合は新規作成
        try:
            # Firebaseに新しいユーザーを作成
            user = auth.create_user(
                email=email,                   # メールアドレス
                email_verified=False,          # メール認証は未確認状態
                password=password,             # パスワード
                display_name=display_name,     # 表示名（オプション）
                phone_number=phone_number      # 電話番号（オプション）
            )
            print("Successfully created new user:", user.uid)
        except ValueError as ve:
            # ユーザー作成に失敗した場合のエラーハンドリング
            print("Error creating user:", ve)
        except Exception as e:
            print("Error: ユーザー作成中に問題が発生しました。")  # 一般的なエラーメッセージで応答

# 使用例
email = "test1@example.com"            # 作成するユーザーのメールアドレス
password = "specified_password123"       # 作成するユーザーのパスワード
display_name = "Test User"               # 作成するユーザーの表示名（オプション）
phone_number = "+819012345672"           # 作成するユーザーの電話番号（オプション、E.164形式）

# ユーザー作成関数を呼び出して新しいユーザーを作成
create_user(email, password, display_name, phone_number)




In [ ]:
# 特定のユーザーまたは全てのユーザー情報を取得する関数
def get_user_info(email=None, uid=None):
    try:
        # 特定のユーザーを指定した場合
        if email:
            user = auth.get_user_by_email(email)
        elif uid:
            user = auth.get_user(uid)
        else:
            print("エラー: メールアドレスまたはUIDを指定してください。")
            return

        # ユーザー情報の表示
        print("ユーザー情報:")
        print("UID:", user.uid)
        print("メールアドレス:", user.email)
        print("表示名:", user.display_name if user.display_name else "未設定")
        print("電話番号:", user.phone_number if user.phone_number else "未設定")

    except auth.UserNotFoundError:
        print("エラー: 指定されたユーザーが見つかりません。")
    except ValueError as ve:
        print("エラー: 無効な入力が検出されました。", ve)
    except Exception as e:
        # その他の予期しないエラーは詳細を表示しない
        print("エラー: ユーザー情報の取得中に問題が発生しました。")
        # ログや管理者にエラー通知を行いたい場合、ここでエラーログに記録可能

# 全てのユーザー情報を取得する関数
def get_all_users():
    try:
        # Firebaseからの全ユーザー情報取得（制限付き）
        all_users = auth.list_users().iterate_all()

        print("全ユーザー情報:")
        for user in all_users:
            print("UID:", user.uid)
            print("メールアドレス:", user.email)
            print("表示名:", user.display_name if user.display_name else "未設定")
            print("電話番号:", user.phone_number if user.phone_number else "未設定")
            print("-" * 30)

    except Exception as e:
        print("エラー: 全ユーザー情報の取得中に問題が発生しました。")
        # 必要に応じてエラーログに記録

# 使用例
# 特定のユーザー情報を取得（メールアドレスかUIDで指定）
get_user_info(email="specified_email@example.com")
# 全てのユーザー情報を取得
get_all_users()



In [ ]:
import re  # パスワードのバリデーションに使用

# ユーザー情報を更新する関数
def update_user_info(uid, display_name=None, phone_number=None, password=None):
    """
    Firebaseのユーザー情報を更新する関数
    uid: 更新対象のユーザーのUID
    display_name: 新しい表示名（オプション）
    phone_number: 新しい電話番号（オプション、E.164形式）
    password: 新しいパスワード（オプション）
    """
    try:
        # 更新内容を格納する辞書を作成
        update_data = {}

        # display_nameが指定されていれば更新内容に追加
        if display_name:
            if len(display_name) > 50:  # 文字数制限
                print("Error: Display name must be 50 characters or less.")
                return
            update_data['display_name'] = display_name

        # phone_numberが指定されていれば、フォーマット確認してから更新内容に追加
        if phone_number:
            if not re.match(r"^\+?\d{10,15}$", phone_number):
                print("Error: Phone number must be in E.164 format, e.g., +819012345678.")
                return
            update_data['phone_number'] = phone_number

        # passwordが指定されていればバリデーション後に追加
        if password:
            if len(password) < 8 or not re.search(r"[A-Za-z]", password) or not re.search(r"\d", password):
                print("Error: Password must be at least 8 characters long and contain both letters and numbers.")
                return
            update_data['password'] = password

        # 辞書に更新項目がある場合のみFirebaseでユーザー情報を更新
        if update_data:
            user = auth.update_user(uid, **update_data)
            print("Successfully updated user:", user.uid)
            print("New Display Name:", user.display_name)
            print("New Phone Number:", user.phone_number)
            print("Password updated" if 'password' in update_data else "Password not updated")
        else:
            print("No updates provided.")

    except auth.UserNotFoundError:
        print("Error: User not found.")
    except ValueError as ve:
        print("Error updating user information due to invalid input:", ve)
    except Exception as e:
        print("Error updating user information:", e)

# 使用例
update_user_info("R1nov2pS4mX3mKlpp1k65qKbcbq1", display_name="テスト太郎", phone_number="+811234561890", password="NewPassword123")


In [ ]:
# ユーザー削除のための関数
def delete_user(uid, confirm=False):
    """
    Firebaseから特定のユーザーを削除する関数
    uid: 削除対象のユーザーのUID
    confirm: 削除確認フラグ（Trueの場合に削除実行）
    """
    try:
        if not confirm:
            # 削除確認メッセージを表示
            print(f"User with UID '{uid}' will be deleted. To confirm, call delete_user with confirm=True.")
            return

        # Firebaseで指定されたUIDのユーザーを削除
        auth.delete_user(uid)
        print("Successfully deleted user:", uid)

        # 削除ログの例（必要に応じて実装）
        # log_deletion(uid)

    except auth.UserNotFoundError:
        print("Error: User not found.")
    except Exception as e:
        print("Error deleting user:", e)

# 使用例
# 削除を確認した場合のみ confirm=True を設定して実行
delete_user("qniJ4XydJ9Q7ovYiWthk2WXH9hE2", confirm=True)


In [ ]:
# ユーザー認証トークンを無効化する関数
def revoke_user_session(uid):
    """
    指定したUIDのユーザーのリフレッシュトークンを無効化し、セッションを失効させる
    uid: セッションを無効化する対象ユーザーのUID
    """
    try:
        # Firebaseでユーザーのリフレッシュトークンを無効化
        auth.revoke_refresh_tokens(uid)
        print("Successfully revoked session for user:", uid)
    except auth.UserNotFoundError:
        print("Error: User not found.")
    except Exception as e:
        print("Error revoking user session:", e)

# 使用例
# UIDを指定して、そのユーザーのセッションを無効化
revoke_user_session("ユーザーのUID")


In [ ]:
# 全ユーザー情報の一覧表示および検索機能
def list_users(search_query=None, search_field="email"):
    """
    全ユーザーの情報を一覧表示し、検索条件が指定された場合はそれに基づいて表示します。
    search_query: 検索するキーワード（オプション）
    search_field: 検索対象のフィールド（"email" または "display_name"、デフォルトは "email"）
    """
    try:
        all_users = auth.list_users().iterate_all()  # 全ユーザーを取得
        print("ユーザー一覧:")

        for user in all_users:
            # 検索条件が指定されている場合、その条件に合うユーザーのみ表示
            if search_query:
                if search_field == "email" and search_query.lower() not in user.email.lower():
                    continue  # 検索キーワードがメールアドレスに含まれていなければスキップ
                elif search_field == "display_name" and user.display_name and search_query.lower() not in user.display_name.lower():
                    continue  # 検索キーワードが表示名に含まれていなければスキップ

            # ユーザー情報の表示
            print("UID:", user.uid)
            print("Email:", user.email)
            print("Display Name:", user.display_name if user.display_name else "未設定")
            print("Phone Number:", user.phone_number if user.phone_number else "未設定")
            print("-" * 30)

    except Exception as e:
        print("Error listing users:", e)

# 使用例
# 全ユーザーを一覧表示
list_users()

# メールアドレスに "example" を含むユーザーを検索
list_users(search_query="example", search_field="email")

# 表示名に "Test" を含むユーザーを検索
list_users(search_query="Test", search_field="display_name")
